<a href="https://colab.research.google.com/github/kohoto/StockAnalysisUsingPython/blob/main/StockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pyquery
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin



     |████████████████████████████████| 911kB 19.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launc

In [88]:
# Set path to the chrome driver directry.
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

from pyquery import PyQuery
import time
import sqlite3

### Inputs

In [116]:
db_name = 'database1.db'
table_name = 'brands'

### Create Database



#### Delete table

In [123]:
def delete_table_in_db(db_file_name,table_name):
    con = sqlite3.connect(db_file_name)
    with con:
        sql = 'DROP TABLE {};'.format(table_name)
        con.execute(sql)

### Create table

In [124]:

  
def create_table_in_db(db_file_name,table_name):
    con = sqlite3.connect(db_file_name)
    with con:
        sql = 'CREATE TABLE "{}" ("symbol"	TEXT, "name"	TEXT, "market"	TEXT, "sector"	TEXT, PRIMARY KEY("symbol"))'.format(table_name)
        con.execute(sql)

#delete_table_in_db(db_name,table_name)
create_table_in_db(db_name,table_name)

Create database for brand names and symbols

### Define functions 

In [125]:
# Get stock data of a single company from website.
def get_name_symbol_market_from_csv():
    # Get names and symbols.
    import pandas as pd
    file = "/content/NASDAQ_short.csv"
    data1 = pd.read_csv(file)

    name = data1['Description']
    symbol = data1['Symbol']
    market = data1['Symbol']
    return name, symbol, market

def get_sector(symbol):
    ## ====== Get data from Trading View web page. ======
    # Option for seenium
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome('chromedriver',options=chrome_options)

    # get tradingview.com
    url = 'https://www.tradingview.com/symbols/NASDAQ-{}/'.format(symbol)
    print(url)
    wd.get(url)
  
    # get element 
    sector = wd.find_element_by_class_name('tv-widget-description__value').text
    return sector

# Generate stock data for many companies.
def get_brand_data_array():
    # Get name, symbol, market, sector arrays.
    [names, symbols, markets] = get_name_symbol_market_from_csv()
    for idx, one_of_symbols in enumerate(symbols):
       sector = get_sector(one_of_symbols)
       if sector:
         brand_data = []
         yield one_of_symbols, names[idx], markets[idx], sector
       time.sleep(1)

# Insert stock data into database.
def insert_brands_to_db(db_file_name):
  # Connect to the database & insert values.
  conn = sqlite3.connect(db_file_name)
  with conn:
     sql = 'INSERT INTO brands(symbol,name,market,sector) ' \
           'VALUES(?,?,?,?)'
     conn.executemany(sql, get_brand_data_array()) ## Something wrong here!! Cannot insert data into db

Main Program

In [126]:
# Main program
insert_brands_to_db(db_name)

https://www.tradingview.com/symbols/NASDAQ-AACG/
https://www.tradingview.com/symbols/NASDAQ-AACQ/
https://www.tradingview.com/symbols/NASDAQ-AACQU/
https://www.tradingview.com/symbols/NASDAQ-AACQW/
